In [1]:
RUNNING_LOCALLY = True

In [2]:
import os

# PLEASE SET YOUR OWN WORKING_DIRECTORY WHEN RUNNING LOCALLY
WORKING_DIRECTORY = "/home/yash/Desktop/Courses/CS2470/Final_Project/working_dir/"

if not RUNNING_LOCALLY:
    os.chdir("/home/yash/")
    print("Current Directory ->", os.getcwd())

    WORKING_DIRECTORY = "/home/yash/working_dir/"

    # Ensure that you are working in the right environment
    !echo $CONDA_PREFIX

LOG_FILE = WORKING_DIRECTORY + "log_file.txt"

def write_to_log(statement, include_blank_line=False):
    try:
        with open(LOG_FILE, "a") as myfile:
            if include_blank_line:
                myfile.write("\n\n" + statement)
            else:
                myfile.write("\n" + statement)
    except:
        # Running this locally may cause errors, and isn't required
        pass

In [3]:
import gym
import time
import numpy as np
from collections import deque
from gym import spaces
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import time
import matplotlib.pyplot as plt


print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2022-05-08 01:08:45.205932: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-08 01:08:45.205951: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/yash/Desktop/Courses/CS2470/Final_Project/repo/proj_env_2/lib/python3.7/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/home/yash/Desktop/Courses/CS2470/Final_Project/repo/proj_env_2/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/home/yash/Desktop/Courses/CS2470/Final_Project/repo/proj

Num GPUs Available:  0


2022-05-08 01:08:47.615174: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-05-08 01:08:47.615214: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-05-08 01:08:47.615232: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (yash-ThinkPad-L390): /proc/driver/nvidia/version does not exist


In [4]:
from matplotlib import pyplot as plt

def render_image(data):
    plt.imshow(data, interpolation='nearest')
    plt.show()

def plot_line(arr):
    x = np.arange(len(arr))
    plt.plot(x, arr)
    plt.show()
# plot_line(np.array([1,2.9,3.1,-4,3.2]))



In [5]:
class ConcatObs(gym.Wrapper):
    def __init__(self, env, k, DEATH_REWARD=0):
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.DEATH_REWARD = DEATH_REWARD
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = \
            spaces.Box(low=0, high=255, shape=((k,) + shp), dtype=env.observation_space.dtype)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):

        # Take k steps at once
        total_reward = 0.0
        done = False
        for i in range(self.k):
            obs, reward, done, info = self.env.step(action)
            total_reward += reward
            self.frames.append(obs)
            
            # only count one live each episode
            if info['lives'] < 4:
                total_reward = total_reward - self.DEATH_REWARD
                done = True
                break

        return self._get_ob(), total_reward, done, info

    def _get_ob(self):
        return np.array(self.frames)

In [6]:
# A bunch of wrappers to get us started, please use these
class ObservationWrapper(gym.ObservationWrapper):
    def __init__(self, env, GRAYSCALE=False, NORMALIZE=False):
        self.GRAYSCALE = GRAYSCALE
        self.NORMALIZE = NORMALIZE
        super().__init__(env)
    
    def observation(self, obs):
        # Normalise observation by 255
        if self.NORMALIZE:
            obs = obs / 255.0
            
        if self.GRAYSCALE:
            obs = tf.image.rgb_to_grayscale(obs)
                    
        image = obs[:,2:-9,8:,:]
        image = tf.image.resize(image,[84,84])
        image = tf.transpose(tf.reshape(image, image.shape[:-1]),perm = [1,2,0])
        return image

class RewardWrapper(gym.RewardWrapper):
    def __init__(self, env, CLIP=False, SCALE=False):
        self.CLIP = CLIP
        self.SCALE = SCALE
        self.reward_dict = {
            30: 0.1,
            60: 0.15,
            80: 0.2,
            100: 0.3,
            500: 0.5,
            -1000: -1,
        }
        super().__init__(env)
    
    def reward(self, reward):
        # Clip reward between 0 to 1
        if self.CLIP:
            # reward = np.clip(reward, 0, 1)
            reward = np.sign(reward)
        if self.SCALE:
            # Creating custom function (dict) to scale rewards
            # Scores based on http://www.atarimania.com/game-atari-2600-vcs-river-raid_s6826.html (Check instructions)
            # tanker: 30
            # helicopter: 60
            # fuel depot: 80
            # jet: 100
            # bridge: 500
            # death: SET in ConcatObs (by default, 0. Currently using 1000)
            reward = self.reward_dict.get(reward, 0)
        return reward
    
class ActionWrapper(gym.ActionWrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def action(self, action):
        return action

class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        super().__init__(env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)


In [42]:
# Configuration paramaters for the whole setup
seed = 42
gamma = 0.99  # Discount factor for past rewards
epsilon = 1.0  # Epsilon greedy parameter
epsilon_min = 0.1  # Minimum epsilon greedy parameter
epsilon_max = 1.0  # Maximum epsilon greedy parameter
epsilon_interval = (
    epsilon_max - epsilon_min
)  # Rate at which to reduce chance of random action being taken
batch_size = 32  # Size of batch taken from replay buffer
# max_steps_per_episode = 100000
max_steps_per_episode = 10000 # CHANGE THIS

In [36]:
# Using the wrappers for the environment
env = gym.make("ALE/Riverraid-v5")
env = ObservationWrapper(RewardWrapper(ActionWrapper(ConcatObs(FireResetEnv(env), k=4, DEATH_REWARD=1000)), CLIP=False, SCALE=True), GRAYSCALE=True, NORMALIZE=True)
obs = env.reset()

In [39]:
hidden_size = 512
num_actions = 18

def create_q_model(input_shape, hidden_size, num_actions):
    inputs = layers.Input(shape=input_shape)

    # Convolutions on the frames on the screen
    layer1 = layers.Conv2D(32, 8, strides=4, activation="relu")(inputs)
    layer2 = layers.Conv2D(64, 4, strides=2, activation="relu")(layer1)
    layer3 = layers.Conv2D(64, 3, strides=1, activation="relu")(layer2)

    layer4 = layers.Flatten()(layer3)

    layer5 = layers.Dense(hidden_size, activation="relu")(layer4)
    action = layers.Dense(num_actions, activation="linear")(layer5)

    return keras.Model(inputs=inputs, outputs=action)


q_net = create_q_model(input_shape=obs.shape, hidden_size=hidden_size, num_actions=num_actions)
q_net_target = create_q_model(input_shape=obs.shape, hidden_size=hidden_size, num_actions=num_actions)

optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

In [43]:
# Experience replay buffers
action_history = []
state_history = []
state_next_history = []
rewards_history = []
done_history = []
episode_reward_history = []
running_reward = 0
episode_count = 0
frame_count = 0
# Number of frames to take random action and observe output
# epsilon_random_frames = 50000
epsilon_random_frames = 2000
# Number of frames for exploration
# epsilon_greedy_frames = 1000000.0
epsilon_greedy_frames = 10000.0
# Maximum replay length
# Note: The Deepmind paper suggests 1000000 however this causes memory issues
# max_memory_length = 100000
max_memory_length = 1000
# Train the model after 4 actions
update_after_actions = 4
# How often to update the target network
# update_target_network = 10000
update_target_network = 100
# Using huber loss for stability
loss_function = keras.losses.Huber()

In [44]:
n_episodes = 2000
while True and episode_count < n_episodes:  # Run until solved
    state = np.array(env.reset())
    episode_reward = 0

    for timestep in range(1, max_steps_per_episode):
        # env.render(); Adding this line would show the attempts
        # of the agent in a pop up window.
        frame_count += 1

        # Use epsilon-greedy for exploration
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Take random action
            action = np.random.choice(num_actions)
        else:
            # Predict action Q-values
            # From environment state
            state_tensor = tf.convert_to_tensor(state)
            state_tensor = tf.expand_dims(state_tensor, 0)
            action_probs = q_net(state_tensor, training=False)
            # Take best action
            action = tf.argmax(action_probs[0]).numpy()

        # Decay probability of taking random action
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)

        # Apply the sampled action in our environment
        state_next, reward, done, _ = env.step(action)
        state_next = np.array(state_next)

        episode_reward += reward

        # Save actions and states in replay buffer
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)
        state = state_next

        # Update every fourth frame and once batch size is over 32
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:

            # Get indices of samples for replay buffers
            indices = np.random.choice(range(len(done_history)), size=batch_size)

            # Using list comprehension to sample from replay buffer
            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array([state_next_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = tf.convert_to_tensor(
                [float(done_history[i]) for i in indices]
            )

            # Build the updated Q-values for the sampled future states
            # Use the target model for stability
            future_rewards = q_net_target.predict(state_next_sample)
            # Q value = reward + discount factor * expected future reward
            updated_q_values = rewards_sample + gamma * tf.reduce_max(
                future_rewards, axis=1
            )

            # If final frame set the last value to -1
            updated_q_values = updated_q_values * (1 - done_sample) - done_sample

            # Create a mask so we only calculate loss on the updated Q-values
            masks = tf.one_hot(action_sample, num_actions)

            with tf.GradientTape() as tape:
                # Train the model on the states and updated Q-values
                q_values = q_net(state_sample)

                # Apply the masks to the Q-values to get the Q-value for action taken
                q_action = tf.reduce_sum(tf.multiply(q_values, masks), axis=1)
                # Calculate loss between new Q-value and old Q-value
                loss = loss_function(updated_q_values, q_action)

            # Backpropagation
            grads = tape.gradient(loss, q_net.trainable_variables)
            optimizer.apply_gradients(zip(grads, q_net.trainable_variables))

        if frame_count % update_target_network == 0:
            # update the the target network with new weights
            q_net_target.set_weights(q_net.get_weights())
            # Log details
            template = "running reward: {:.2f} at episode {}, frame count {}"
            print(template.format(running_reward, episode_count, frame_count))

        # Limit the state and reward history
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        if done:
            break

    # Update running reward to check condition for solving
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    episode_count += 1

    if running_reward > 10:  # Condition to consider the task solved
        # Considering solved because the game has achieved a score of 10 without dying
        print("Solved at episode {}!".format(episode_count))
        break

2022-05-08 00:36:36.077452: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26214400 exceeds 10% of free system memory.
2022-05-08 00:36:36.082155: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26214400 exceeds 10% of free system memory.
2022-05-08 00:36:36.221915: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26214400 exceeds 10% of free system memory.
2022-05-08 00:36:36.225159: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26214400 exceeds 10% of free system memory.
2022-05-08 00:36:36.353435: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 26214400 exceeds 10% of free system memory.


running reward: 2.67 at episode 3, frame count 100
running reward: 2.00 at episode 8, frame count 200
running reward: 2.25 at episode 12, frame count 300
running reward: 1.88 at episode 17, frame count 400
running reward: 2.00 at episode 21, frame count 500
running reward: 1.96 at episode 25, frame count 600
running reward: 1.90 at episode 30, frame count 700
running reward: 2.00 at episode 34, frame count 800
running reward: 2.16 at episode 37, frame count 900
running reward: 2.33 at episode 39, frame count 1000
running reward: 2.28 at episode 43, frame count 1100
running reward: 2.24 at episode 46, frame count 1200
running reward: 2.20 at episode 51, frame count 1300
running reward: 2.16 at episode 56, frame count 1400
running reward: 2.17 at episode 60, frame count 1500
running reward: 2.12 at episode 64, frame count 1600
running reward: 2.13 at episode 69, frame count 1700
running reward: 2.15 at episode 72, frame count 1800
running reward: 2.16 at episode 75, frame count 1900
runn

KeyboardInterrupt: 

Compare with random model<br>
> running reward: 2.46 at episode 399, frame count 10400 (this is the average of the last 100 games)

In [11]:
# Using the wrappers for the environment
env = gym.make("ALE/Riverraid-v5")
env = ObservationWrapper(RewardWrapper(ActionWrapper(ConcatObs(FireResetEnv(env), k=4, DEATH_REWARD=1000)), CLIP=False, SCALE=True), GRAYSCALE=True, NORMALIZE=True)
obs = env.reset()

num_actions = 18
n_episodes_random = 1000
episode_reward_history = np.zeros(n_episodes_random)
episode_count = 0
while episode_count < n_episodes_random:
    state = np.array(env.reset())
    episode_reward = 0

    done = False
    while not done:

        action = np.random.choice(num_actions)
        # Apply the sampled action in our environment
        state_next, reward, done, _ = env.step(action)
        episode_reward += reward
        if done:
            break

    episode_reward_history[episode_count] = episode_reward
    episode_count += 1
    if episode_count%100 == 0:
        print(episode_count)

100
200
300
400
500
600
700
800
900
1000


In [12]:
def describe_arr(arr):
    return np.min(arr), np.max(arr), np.mean(arr), np.std(arr)

describe_arr(episode_reward_history)

(-1.0, 1.7999999999999998, -0.65195, 0.31050555148016273)

In [ ]:
# Record video of random to check whether it runs for a single life or not

Record video to check